### Import dependencies

In [1]:
import database
from data_api import youtube
from pprint import pprint
import os


In [5]:
DEVELOPER_KEY = os.getenv('YT_API_KEY')

### Import Google Data API Key and Initiate Youtube API Class Instance
1. Create Youtube Data API key as per the instructions at: https://developers.google.com/youtube/v3/getting-started \

2. Create `.env` file containing the key as follows: `YT_API_KEY=[key]`

In [ ]:
db = database.yt_sentiment()
db.update_categories()

In [3]:
yt = youtube(DEVELOPER_KEY)

KeyboardInterrupt: 

In [6]:
response = yt.popular(28, 'GB')
pprint(response)